<a href="https://colab.research.google.com/github/ramanathanlab/genslm/blob/main/examples/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# NOTE: You may need to run this twice due to a pip dependency conflict
# !pip install git+https://github.com/ramanathanlab/genslm

  Cloning https://github.com/ramanathanlab/genslm to /private/var/folders/kb/4rj6v4lj55b0h_wrblbl2pt80000gp/T/pip-req-build-rug63m21
  Running command git clone --filter=blob:none --quiet https://github.com/ramanathanlab/genslm /private/var/folders/kb/4rj6v4lj55b0h_wrblbl2pt80000gp/T/pip-req-build-rug63m21
  Resolved https://github.com/ramanathanlab/genslm to commit e4fbf3b8e641150d708c18e12d551de8ed0cae1c
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/maxzvyagin/transformers to /private/var/folders/kb/4rj6v4lj55b0h_wrblbl2pt80000gp/T/pip-install-7xj2wut8/transformers_6c83181bd23742ddb3229187a99a18fb
  Running command git clone --filter=blob:none --quiet https://github.com/maxzvyagin/transformers /private/var/folders/kb/4rj6v4lj55b0h_wrblbl2pt80000gp/T/pip-install-7xj2wut8/transformers_6c83181bd23742ddb3229187a99a18fb
  Resolved https://github.com/maxzvyagin/transformers to commit ffd5aba0ad41a1ebd1897a77f6a3782fc2d75e1f
  Installing build dependencies ... done
 

In [3]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!ls ~/projects/exascale/genslm/models/25M

patric_25m_epoch01-val_loss_0.57_bias_removed.pt


In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from genslm import GenSLM, SequenceDataset

# Load model
# model = GenSLM("genslm_25M_patric", model_cache_dir="/content/gdrive/MyDrive")
model = GenSLM("genslm_25M_patric", model_cache_dir="/users/jdtibochab/projects/exascale/genslm/models/25M")

model.eval()

# Select GPU device if it is available, else use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Input data is a list of gene sequences
sequences = [
    "ATGAAAGTAACCGTTGTTGGAGCAGGTGCAGTTGGTGCAAGTTGCGCAGAATATATTGCA",
    "ATTAAAGATTTCGCATCTGAAGTTGTTTTGTTAGACATTAAAGAAGGTTATGCCGAAGGT",
]

dataset = SequenceDataset(sequences, model.seq_length, model.tokenizer)
dataloader = DataLoader(dataset)

# Compute averaged-embeddings for each input sequence
embeddings = []
with torch.no_grad():
    for batch in dataloader:
        outputs = model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device),
            output_hidden_states=True,
        )
        # outputs.hidden_states shape: (layers, batch_size, sequence_length, hidden_size)
        # Use the embeddings of the last layer
        emb = outputs.hidden_states[-1].detach().cpu().numpy()
        # Compute average over sequence length
        emb = np.mean(emb, axis=1)
        embeddings.append(emb)

# Concatenate embeddings into an array of shape (num_sequences, hidden_size)
embeddings = np.concatenate(embeddings)
embeddings.shape

/users/jdtibochab/.local/share/mamba/envs/genslm-py37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Tokenizing...: 100%|██████████| 2/2 [00:00<00:00, 535.19it/s]


(2, 512)

In [ ]:
# NOTE: This is not the best performance you can get. For a scalable implementation,
# refer to genslm.cmdline.run_inference for an example of how to utilize multiple
# GPUs for parallel inference.